In [ ]:
#! /usr/bin/env Rscript
'Time how long it takes to run cellchat given some samples
Usage:
    Rscript 04_cellchat_timing.r --number=<int> --group=<bool> [--seed=<int>]
    
Options:
    -h --help  Show this screen.
    --number=<int>  number of samples to include
    --group=<bool>  whether to group by context (1) or keep separated by samples (0)
    --seed=<int>  set a seed for random graph generation 
' -> doc

In [1]:
suppressPackageStartupMessages({
    library(CellChat, quietly = T)
    library(patchwork, quietly = T)
    library(RhpcBLASctl, quietly = T)
    library(Matrix, quietly = T)
    library(docopt, quietly = T)
    library(rhdf5, quietly = T)
    library(stringr, quietly = T)
})
options(stringsAsFactors = FALSE)
# expression_data_path = '/data2/hratch/immune_CCI/covid/covid_atlas/interim/umi_for_timing/'
input_path = '/data2/hratch/immune_CCI/covid/covid_atlas/interim/timing_inputs/'

RhpcBLASctl::blas_set_num_threads(1) # no multithreading

# parameters
cell_grouper<-'majorType'
type_<-'functional'

Warning message:
“package ‘docopt’ was built under R version 4.0.5”


In [ ]:
opts = docopt(doc)
for (opt in c('number', 'group', 'seed')){
    val = opts[[opt]]
    if (!is.null(val)){
        opts[[opt]] = as.numeric(val)
        }}

number<-opts[['number']]
group<-opts[['group']]
seed<-opts[['seed']]
set.seed(seed)

Load data:

In [3]:
samples<-read.csv(paste0(input_path, 'samples_for_timing.csv'))
sample.names<-unlist(strsplit(samples[samples$n_samples == number, 'sample_names'], '; '))

# load LR pairs
# filter for the LR pairs used by Tensor cell2cell
# lr_pairs<-read.csv(paste0(input_data_path,'Tensor-cell2cell-LRpairs.csv'))
# lr_pairs<-lr_pairs$interaction_name
humandb<-CellChatDB.human
# humandb$interaction<-CellChatDB.human$interaction[CellChatDB.human$interaction$interaction_name %in% lr_pairs, ] 
# saveRDS(humandb, paste0(output_results_path, 'humandb.rds'))

# load metadata
md.cell <- read.csv(paste0(input_path,'metadata_for_timing.csv'), row.names = 1)
sample_context_map<-readRDS(paste0(input_path, 'sample_context_map.rds'))
contexts<-unique(sample_context_map)

In [6]:
# load the UMI counts
read_sample_h5<-function(sn){
    counts<-h5read(paste0(input_path, 'umi_per_sample.h5'), sn)
    count<-counts[[4]]
    colnames(count)<-counts[[2]]
    rownames(count)<-sapply(counts[[1]], function(x) str_replace_all(x, '-', '.')) 
    return(count)
}


if (!group){                             
    counts<-lapply(setNames(sample.names, sample.names), function(sn) read_sample_h5(sn))
                                  
}else{ # group by context
    by.context<-lapply(setNames(contexts, contexts), function(context) names(sample_context_map[sample.names][sample_context_map[sample.names] == context]))
    
    group_by_context<-function(context){
        sns<-by.context[[context]]       
        counts<-lapply(sns, function(sn) read_sample_h5(sn))    
        counts<-do.call(cbind, counts)
        return (counts)
    }
    counts<-lapply(setNames(contexts, contexts), function(context) group_by_context(context))                   
                    
}
                   
# filter for intersection of cell types across samples/contexts - to make comparable with Tensor-cell2cell
# cell.types<-Reduce(intersect, lapply(sample.names, function(sn) unique(md.cell[md.cell$sampleID == sn, cell_grouper])))
cell.types<-Reduce(intersect, lapply(counts, function(df) unique(md.cell[colnames(df), cell_grouper])))
cell.ids<-rownames(md.cell[md.cell[[cell_grouper]] %in% cell.types, ])   
# counts<-lapply(setNames(counts, counts), function(df) df[,colnames(df) %in% cell.ids])  # lapply version slow for some reason, see for loop below                    
for (n in names(counts)){
    df<-counts[[n]]
    counts[[n]]<-df[,colnames(df) %in% cell.ids]
}

Run cellchat:

In [13]:
#' Rank the similarity of the shared signaling pathways based on their joint manifold learning
#'
#' @param object CellChat object
#' @param slot.name the slot name of object that is used to compute centrality measures of signaling networks
#' @param type "functional","structural"
#' @param comparison1 a numerical vector giving the datasets for comparison. This should be the same as `comparison` in `computeNetSimilarityPairwise`
#' @param comparison2 a numerical vector with two elements giving the datasets for comparison.
#'
#' If there are more than 2 datasets defined in `comparison1`, `comparison2` can be defined to indicate which two datasets used for computing the distance.
#' e.g., comparison2 = c(1,3) indicates the first and third datasets defined in `comparison1` will be used for comparison.
#' @import ggplot2
#' @importFrom methods slot
#' @return
#' @export
#'
#' @examples
rankSimilarity_ <- function(object, slot.name = "netP", type = c("functional","structural"), comparison1 = NULL,  
                           comparison2 = c(1,2)) {
  type <- match.arg(type)

  if (is.null(comparison1)) {
    comparison1 <- 1:length(unique(object@meta$datasets))
  }
  comparison.name <- paste(comparison1, collapse = "-")
  cat("Compute the distance of signaling networks between datasets", as.character(comparison1[comparison2]), '\n')
  comparison2.name <- names(methods::slot(object, slot.name))[comparison1[comparison2]]

  Y <- methods::slot(object, slot.name)$similarity[[type]]$dr[[comparison.name]]
  group <- sub(".*--", "", rownames(Y))
  data1 <- Y[group %in% comparison2.name[1], ]
  data2 <- Y[group %in% comparison2.name[2], ]
  rownames(data1) <- sub("--.*", "", rownames(data1))
  rownames(data2) <- sub("--.*", "", rownames(data2))

  pathway.show = as.character(intersect(rownames(data1), rownames(data2)))
  data1 <- data1[pathway.show, ]
  data2 <- data2[pathway.show, ]
  euc.dist <- function(x1, x2) sqrt(sum((x1 - x2) ^ 2))
  dist <- NULL
  for(i in 1:nrow(data1)) dist[i] <- euc.dist(data1[i,],data2[i,])
  df <- data.frame(name = pathway.show, dist = dist, row.names = pathway.show)
  df <- df[order(df$dist), , drop = F]
  df$name <- factor(df$name, levels = as.character(df$name))

  return(df)
}

In [16]:
# create cellchat object for each sample or sample.name
covid.list<-list()
for (sample.name in names(counts)){
    # loop through each sample.name and create a cell type future
    expr<-CellChat::normalizeData(counts[[sample.name]])
    cellchat<-createCellChat(object = as(expr, "dgCMatrix"), meta = md.cell[colnames(expr),], 
                                   group.by = cell_grouper)
    cellchat@DB <- humandb # human organism

    cellchat <- subsetData(cellchat) # subset the expression data of signaling genes, assign to @data.signalling 
    cellchat <- identifyOverExpressedGenes(cellchat)
    cellchat <- identifyOverExpressedInteractions(cellchat) # generate @ LR slot used by computeCommunProb
    cellchat <- projectData(cellchat, PPI.human) # shallow sequencing depth
    
    cellchat <- computeCommunProb(cellchat, raw.use = F, type = 'triMean', trim = NULL, seed.use = seed, 
                                 population.size = F) 
    
    # The functional similarity analysis requires the same cell population composition between two datasets.
    cellchat <- filterCommunication(cellchat, min.cells = 10)
    cellchat <- computeCommunProbPathway(cellchat)
    covid.list[[sample.name]]<-cellchat
}

# merge and analyze
cellchat <- mergeCellChat(covid.list, add.names = names(covid.list))
cellchat <- computeNetSimilarityPairwise(cellchat, type = type_)
cellchat <- netEmbedding(cellchat, type = type_)
cellchat <- netClustering(cellchat, type = type_,  do.parallel = F, do.plot = F)
# Manifold learning of the signaling networks for datasets 
pairwise_comparisons<-sapply(as.data.frame(combn(seq(1:length(covid.list)),2)), 
                         function(x) as.numeric(x), simplify = F) # pairwise combination of elements

for (pc in names(pairwise_comparisons)){
    path.dist<-rankSimilarity_(cellchat, type = type_, comparison1 = 1:length(covid.list),
                               comparison2 = pairwise_comparisons[[pc]]) 
}                      

Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  B CD4 CD8 DC Mega Mono NK Plasma 
The cell-cell communication related with the following cell groups are excluded due to the few number of cells:  Mega Plasma 


Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  B CD4 CD8 DC Mega Mono NK Plasma 
The cell-cell communication related with the following cell groups are excluded due to the few number of cells:  Mega Plasma 


Create a CellChat object from a data matrix

Set cell identities for the new CellChat object



The cell groups used for CellChat analysis are  B CD4 CD8 DC Mega Mono NK Plasma 
The cell-cell communication related with the following cell groups are excluded due to the few number of cells:  Mega Plasma 


Merge the following slots: 'data.signaling','net', 'netP','meta', 'idents', 'var.features' , 'DB', and 'LR'.



Compute signaling network similarity for datasets 1 2 3 


Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg + Ph)/2) - (entropia(pg) + entropia(ph))/2)/log(2)):
“NaNs produced”
Warning message in sqrt((entropia((Pg

Manifold learning of the signaling networks for datasets 1 2 3 
Classification learning of the signaling networks for datasets 1 2 3 
Compute the distance of signaling networks between datasets 1 2 
Compute the distance of signaling networks between datasets 1 3 
Compute the distance of signaling networks between datasets 2 3 
